In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Đọc dữ liệu từ file CSV
file_path = './TrafficTwoMonth.csv'
df = pd.read_csv(file_path)

# Hiển thị thông tin cơ bản của dữ liệu
print(df.info())

# Tiền xử lý dữ liệu
df['Time'] = pd.to_datetime(df['Time'], format='%I:%M:%S %p')
df['Time'] = df['Time'].apply(lambda x: x.timestamp())  # Convert time to timestamp

# Tách dữ liệu thành features và target
X = df[['Time']]
y = df['Total']

# Chuẩn hóa đặc trưng
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Xây dựng mô hình RNN
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))

# Biên dịch mô hình
model.compile(optimizer='adam', loss='mean_squared_error')

# Chỉ dừng sớm để tránh overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reshape dữ liệu cho LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Đánh giá mô hình trên tập kiểm tra
mse = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {mse}')

# Thực hiện dự đoán
predictions = model.predict(X_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Time               5952 non-null   object
 1   Date               5952 non-null   int64 
 2   Day of the week    5952 non-null   object
 3   CarCount           5952 non-null   int64 
 4   BikeCount          5952 non-null   int64 
 5   BusCount           5952 non-null   int64 
 6   TruckCount         5952 non-null   int64 
 7   Total              5952 non-null   int64 
 8   Traffic Situation  5952 non-null   object
dtypes: int64(6), object(3)
memory usage: 418.6+ KB
None
Epoch 1/50
149/149 [==============================] - 3s 6ms/step - loss: 15021.4746 - val_loss: 14738.7812
Epoch 2/50
149/149 [==============================] - 1s 4ms/step - loss: 14662.0059 - val_loss: 14124.2266
Epoch 3/50
149/149 [==============================] - 1s 5ms/step - loss: 13711.8506 - val_loss: 12840.6